In [19]:
!pip -q install datasets langchain-groq

## Hello Groq!

In [20]:
GROQ_API_KEY="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"

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import pprint
pp = pprint.PrettyPrinter(indent=4)

chat = ChatGroq(temperature=0, model_name="llama3-70b-8192", api_key=GROQ_API_KEY)

In [22]:
system = "You are a helpful assistant."
human = "{text}"
prompt_template = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt_template | chat
response = chain.invoke({"text": "Hello World!."})
pp.pprint(response.__dict__)

{   'additional_kwargs': {},
    'content': "Hello there! It's great to meet you! I'm your helpful "
               'assistant, here to assist you with any questions, tasks, or '
               "topics you'd like to discuss. What brings you here today?",
    'example': False,
    'id': 'run-b38111c4-60c3-47c5-a975-f2d1e4f62c4f-0',
    'invalid_tool_calls': [],
    'name': None,
    'response_metadata': {   'finish_reason': 'stop',
                             'logprobs': None,
                             'model_name': 'llama3-70b-8192',
                             'system_fingerprint': 'fp_2f30b0b571',
                             'token_usage': {   'completion_time': 0.110525817,
                                                'completion_tokens': 41,
                                                'prompt_time': 0.006959783,
                                                'prompt_tokens': 24,
                                                'queue_time': None,
                      

- The response from the LLM is contained in `content`


In [5]:
print(response.content)

Hello there! It's great to meet you! I'm your helpful assistant, here to assist you with any questions, tasks, or topics you'd like to discuss. What brings you here today?


## Load GSM8K dataset using 🤗Huggingface's datasets library.

In [6]:
from datasets import load_dataset

gsm8k = load_dataset("gsm8k", "main")['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
print("Question:")
for l in gsm8k['question'][0].split("."):
    print(l)
print("="*100)
print("Answer:")
print(gsm8k['answer'][0])

Question:
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
Answer:
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


In [ ]:
import random
random.seed(0)

def construct_direct_prompt(num_exemplars):
    # Load train set of GSM8K
    gsm8k = load_dataset("gsm8k", "main")['train']

    sampled_indices = random.sample([i for i in range(len(gsm8k['question']))], num_exemplars)


    instruction = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale."

    # Constructing a prompt with few-shot demonstrations from GSM8K
    prompt = instruction
    for i in range(num_exemplars):
        cur_question = gsm8k['question'][i]
        cur_answer = gsm8k['answer'][i].split("####")[-1].strip()
        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += f"\n[Example {num_exemplars+1}]\n"
    prompt += "Question:\n{question}\nAnswer:"

    # Write the prompt to a .txt file
    with open(f"direct_prompt_{num_exemplars}shot.txt", "w") as f:
        f.write(prompt)



In [ ]:
construct_direct_prompt(0)
construct_direct_prompt(3)

In [ ]:
with open("direct_prompt_3shot.txt", "r") as f:
    prompt = f.read()

## Check out the model input which is constructed by formatting the 3-shot prompt (direct prompting)

```Instruction:
Solve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.
[Example 1]
Question:
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Answer:72

[Example 2]
Question:
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
Answer:10

[Example 3]
Question:
Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
Answer:5

[Example 4]
Question:
Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
```

In [ ]:
cur_model_input = prompt.format(question=gsm8k['question'][0])
print(cur_model_input)

Instruction:
Solve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.
[Example 1]
Question:
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Answer:72

[Example 2]
Question:
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
Answer:10

[Example 3]
Question:
Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
Answer:5

[Example 4]
Question:
Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How mu

In [9]:
from tqdm import tqdm
import json
import re

results_collected = []
pass_collected = []
VERBOSE = False
def parse_model_responses(text):
    # Enhancing regex to capture numbers possibly associated with units or other contexts
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, text, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    # If no matches found in previous patterns, attempt to retrieve simpler numeric or monetary values
    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, text, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    if VERBOSE:
        print(f"Results found: {results}")
    return results[-1] if results else None

for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_answer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt.format(question=cur_question)
    result = chain.invoke({"text":cur_model_input}).content
    if "Error:" in result:
        print(result)
        break

    cur_prediction = parse_model_responses(result)
    if VERBOSE:
        print("Raw response:", result)
        print("Predicted answer:",cur_prediction)
        print("Reference answer:",cur_answer)

    pass_collected.append(cur_prediction.strip().replace("$", "0") == cur_answer)
    results_collected.append({"question": cur_question, "answer": cur_answer, "prediction": cur_prediction})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")

    if VERBOSE and i == 10:
        break

    with open("result_3shot_direct.json", "w") as f:
        json.dump(results_collected, f, indent=4)

  0%|          | 0/50 [00:00<?, ?it/s]


NameError: name 'prompt' is not defined

## Let's apply ⛓Chain-of-Thought Prompting!

In [ ]:
gsm8k[0]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [ ]:

def construct_cot_prompt(num_exemplars):

    # Load train set of GSM8K
    gsm8k = load_dataset("gsm8k", "main")['train']

    sampled_indices = random.sample([i for i in range(len(gsm8k['question']))], num_exemplars)


    instruction = "Instruction:\nGenerate the answer for the given mathematical question with step-by-step rationale toward the answer. Provide your final answer based on the rationale using an identifier '####'."

    # Constructing a prompt with few-shot demonstrations from GSM8K
    prompt = instruction
    for i in range(num_exemplars):
        cur_question = gsm8k['question'][i]
        cur_answer = gsm8k['answer'][i]
        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Output:{cur_answer}\n"

    prompt += f"\n[Example {num_exemplars+1}]\n"
    prompt += "Question:\n{question}\n"

    # Write the prompt to a .txt file
    with open(f"cot_prompt_{num_exemplars}shot.txt", "w") as f:
        f.write(prompt)


In [ ]:
import random
random.seed(0)

def construct_cot_prompt(num_exemplars):
    # Load train set of GSM8K
    gsm8k = load_dataset("gsm8k", "main")['train']

    sampled_indices = random.sample([i for i in range(len(gsm8k['question']))], num_exemplars)


    instruction = "Instruction:\nSolve the given simple math problem with explanation. The final solution should be denoted with identifier '###'."

    # Constructing a prompt with few-shot demonstrations from GSM8K
    prompt = instruction
    for i in range(num_exemplars):
        cur_question = gsm8k['question'][i]
        cur_answer = gsm8k['answer'][i]
        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Output:{cur_answer}\n"

    prompt += f"\n[Example {num_exemplars+1}]\n"
    prompt += "Question:\n{question}\n"

    # Write the prompt to a .txt file
    with open(f"cot_prompt_{num_exemplars}shot.txt", "w") as f:
        f.write(prompt)



## Construct 0-, and 3-shot CoT prompts

In [ ]:
construct_cot_prompt(0)
construct_cot_prompt(5)

## Load your CoT prompt

In [11]:
#with open(cot_3shot.txt", "r") as f:
#    cot_prompt = f.read()

cot_prompt = """instruction: 주어진 문제를 읽고 예시를 참고해 풀이과정과 함께 답안을 서술하시오. 답안은 #### 으로 풀이와 구분하시오.

[Example 1]
Question:
컴퓨터 사용자 갑, 을, 병, 정의 아이디와 패스워드를 다음 규칙으로 정하고자 한다.

◦아이디는 apple, banana, cherry, durian 중 하나이다.
◦패스워드는 apple, banana, cherry, durian 중 하나이다.
◦하나의 아이디를 두 명 이상이 같이 쓸 수 없다.
◦하나의 패스워드를 두 명 이상이 같이 쓸 수 없다.
◦사용자의 아이디와 패스워드는 같을 수 없다.
◦을의 아이디는 cherry이다.
◦정의 패스워드는 durian이다.
◦병의 아이디는 아이디가 banana인 사용자의 패스워드와 같다.

정의 아이디는 apple인가?

Output:정의 아이디가 'apple'이라고 가정하자. 그렇다면 병의 아이디는 'durian'이 된다.
다른 조건에 따라 갑의 아이디는 'banana'가 되며, 갑의 패스워드는 'durian'이 된다.
그러나 이때 갑과 병의 패스워드가 일치하는 모순이 발생한다.
#### False

[Example 2]
Question:
A반 4명, B반 3명, C반 3명, D반 2명으로 구성된 동아리를 세 개의 팀으로 나누는데, 다음 조건을 만족한다.

◦각 학생은 어느 한 팀에만 포함된다.
◦각 팀은 최소한 3개의 반의 학생을 포함한다.
◦특정 반의 학생 전체를 포함한 팀은 없다.

각 팀의 학생의 수가 모두 같을 수 있는가?

Output:각 팀에 (a, a, b, c), (a. b. c. d). (a, b, c, d) 와 같이 배정할 수 있다. 이경우 각 팀의 학생수가 같다.
#### True

[Example 3]
Question:
<규정>
제1조주차수요를 유발하는 건축물 등(‘시설물’)을 건축하거나 설치하려는 자는 <표>의 용도별 설치기준에 따라 부설주차장을 설치하여야 한다.
제2조①부설주차장에 설치된 기계식주차장치가 노후ㆍ고장 등의 이유로 작동이 불가능하거나 안전상 철거가 불가피한 경우 이를 철거할 수 있다.
②시설물의 소유자는 제1항에 따라 기계식주차장치를 철거함으로써 제1조에 따른 부설주차장의 설치기준에 미달하게 되는 경우에는 부설주차장을 추가로 설치하여야 한다.
③구청장은 제1항에 따라 기계식주차장치를 철거하는 경우 <표>의 부설주차장 설치기준을 2분의 1로 완화하여야 한다.
④제3항에 의해 완화된 설치기준에 따라 부설주차장을 설치한 이후 해당 시설물이 증축되거나 부설주차장 설치기준이 강화되는 용도로 변경될 때에는 그 증축 또는 용도변경하는 부분에 대해서만 <표>의 부설주차장 설치기준을 적용한다.
제3조시설물의 용도를 변경하는 경우 용도변경 시점의 부설주차장 설치기준에 따라 변경 후 용도의 최소 주차대수를 갖추도록 부설주차장을 설치하여야 한다.
<표>

시설물의 용도 설치기준(최소 주차대수)
위락시설 시설면적 100㎡당 1대
판매시설 시설면적 150㎡당 1대

<사례>
갑은 판매시설로 사용되는 시설면적 6,000㎡의 시설물의 소유자이다. 40대를 수용하는 기존 기계식주차장치가 고장으로 작동이 불가능하자 갑은 이 기계식주차장치를 전부 철거하고, 구청장으로부터 부설주차장 주차기준을 2분의 1로 완화 적용받아 20대를 수용하는 부설주차장을 설치하였다. 갑은 이 시설물의 시설면적 중 3,000㎡를 위락시설로 용도변경하려 한다.

<규정>에 따라 <사례>의 갑이 추가로 갖추어야 할 최소 주차대수는?

Output:변경된 3000 제곱미터의 위락시설이에 대한 필수 주차대수는 다음과 같이 계산할 수 있다.
3000 / 100 - 20 / 2 = 30 - 10 = 20
### 20

[Example 4]
Question:{question}"""

In [ ]:
# cur_model_input = cot_prompt.format(question=gsm8k['question'][0])
# print(cur_model_input)

AttributeError: 'list' object has no attribute 'format'

In [23]:
from tqdm import tqdm
import json
import re

VERBOSE=True
results_collected = []
pass_collected = []

test = {}
test['question'] = ['''
<규정>
(1)형벌 중 중형에는 다음 여섯 등급이 있다.

1등급 사형
2등급 노역 5년 후 3천 리 밖으로 유배
3등급 3천 리 밖으로 유배
4등급 2천 리 밖으로 유배
5등급 노역 3년 6개월
6등급 노역 3년

(2)사람을 때려 재물을 빼앗은 자는 3천 리 밖으로 유배한다.
(3)다른 사람의 범죄를 도운 자는 범죄를 저지른 자보다 한 등급을 감경하여 처벌한다.
(4)자신을 체포하려는 포졸을 때려 상해를 입힌 자의 형벌은 네 등급을 가중한다.
(5)탈옥한 자의 형벌은 세 등급을 가중한다.
(6)자수한 자의 형벌은 세 등급을 감경한다.
(7)1∼3등급에서 형을 감경하는 경우 3등급, 4등급은 하나의 등급으로 취급한다. 가령 2등급에서 두 등급을 감경하면 5등급이다.
(8)3∼6등급에서 형을 가중하는 경우 2등급이 상한이다.
(9)(3)∼(6)의 형벌 가중ㆍ감경 사유 중 두 개 이상에 해당하면, 해당 사유 모두를 (3), (4), (5), (6)의 순서대로 적용한다.
<사례>
갑이 을을 때려 재물을 빼앗는 동안 병은 갑을 위하여 망을 보아주었다. 도망쳐 숨어 지내던 병은 포졸 정의 눈에 띄어 체포될 위기에 처하자 그를 때려 상해를 입히고 달아났다. 이후 병은 관아에 자수하고 갇혀 있던 중 탈옥하였다.
<규정>에 따라 <사례>의 병이 받을 형벌은? 만약 답이 1년 6개월이라면 1.5, 1년이라면 1과 같이 십진수로 표현하시오.''',
    '''키패드에 0 부터 9 까지 숫자가 적혀있다.
숫자가 적혀 있는 두 사각형이 한 변을 서로 공유할 때 두 숫자가 ‘인접’한다고 하자. 서로 다른 6개의 숫자를 한 번씩만 사용하여 만든 암호에 대하여 다음 정보가 알려져 있다.
◦4와 인접한 숫자 중 두 개가 사용되었다.
◦6이 사용되었다면 9도 사용되었다.
◦8과 인접한 숫자 중 한 개만 사용되었다.

5, 6, 7 중에 사용된 숫자는 한 개인가?''',
    '''<X법>
제1조(재해 등에 대한 보상) 국가의 업무 수행 중에 부상을 입거나 사망하면 재해 보상금을 지급하고, 치료로 인하여 생업에 종사하지 못하면 그 기간 동안 휴업 보상금을 지급한다. 다만, 다른 법령에 따라 국가의 부담으로 같은 종류의 보상금을 받은 자에게는 그 보상금에 상당하는 금액은 지급하지 아니한다.
제2조(재해 보상금의 지급) ① 제1조에 따른 재해 보상금은 사망 보상금과 장애 보상금으로 구분하며, 그 지급액은 다음과 같다.
1.사망 보상금은 고용노동부에서 공표하는 전체 산업체 월평균임금총액(사망한 해의 전년도를 기준으로 한다)의 36배에 상당하는 금액
2.장애 보상금은 장애등급에 따라 다음과 같이 정한다.
가～마. 장애등급 1급～5급: (생략)
바. 장애등급 6급: 사망 보상금의
제3조(휴업 보상금의 지급) 제1조에 따른 휴업 보상금은 통계청이 매년 공표하는 도시 및 농가가계비를 평균한 금액(전년도를 기준으로 한다)의 100분의 60에 해당하는 금액을 월 30일을 기준(31일이 말일인 경우에도 같다)으로 하여 1일 단위로 계산한 금액에 치료로 인하여 생업에 종사하지 못한 기간의 일수를 곱한 금액으로 한다.

<사례>
자영업자 갑은 2016년 8월 예비군 훈련 중 자신의 과실 없이 사고로 부상을 입어 60일간의 입원 치료로 생업에 종사하지 못하였고, 장애등급 6급 판정을 받았다. 갑의 월평균 수입은 360만 원이고, 고용노동부에서 공표하는 전체 산업체 월평균임금총액은 2015년 240만 원, 2016년 250만 원이다. 통계청이 공표하는 도시 및 농가가계비를 평균한 금액은 2015년 월 100만 원, 2016년 월 120만 원이다. 한편, 갑은 위 부상과 관련하여 X법이 아닌 다른 법령에 따라 국가로부터 재해 보상금으로 400만 원을 지급받았다.
<X법>을 <사례>에 적용할 때 갑이 지급받을 수 있는 보상금의 총합은?''',
    '''어떤 학과의 졸업 예정자 갑～무에 대해 다음이 알려졌다.

◦취업을 한 학생은 졸업평점이 3.5 이상이거나 외국어 인증시험에 합격했다.
◦인턴 경력이 있는 학생들 중 취업박람회에 참가하지 않은 학생은 아무도 없었다.
◦졸업평점이 3.5 이상이고 취업박람회에 참가한 학생은 모두 취업을 했다.
◦외국어 인증시험에 합격하고 인턴 경력이 있는 학생들은 모두 취업을 했다.

인턴 경력이 있고 졸업평점이 3.5 이상인 무는 취업을 했는가?''',
    '''여러 개의 프로그램이 동시에 실행되면서 같은 작업을 수행하는 병렬 프로그래밍에서, 각 프로그램이 사용하는 데이터는 일정한 메모리 영역에 저장되고 공유된다. 프로그램 P1∼P4와 이들이 사용하는 메모리 영역 M1∼M4에 대하여 다음이 성립한다.

◦P1∼P4만이 실행되고 각 프로그램은 M1∼M4를 사용한다. 각 프로그램은 적어도 1개 이상의 메모리 영역을 사용하고 어떤 프로그램에 의해서도 사용되지 않는 메모리 영역은 없다.
◦메모리 영역은 M1∼M4의 순서대로 일렬로 연결되어 있다.
◦전체 프로그램이 사용하는 메모리 영역의 개수의 합은 최대 6이다.
◦어떤 프로그램도 연속되는 2개의 메모리 영역을 사용할 수 없다.
◦P1은 2개의 메모리 영역을 사용한다.
◦P2는 M2를 사용한다.
◦P4는 P2가 사용하는 메모리 영역을 1개 이상 공유한다.

M2가 3개의 프로그램에 의해서 사용될 수도 있는가?''',
    '''<규정>
종업원은 직무와 관련하여 한 발명에 대하여 기여도에 따라 다음과 같이 보상을 받을 권리를 가진다.
(1)회사가 직무발명에 대한 특허출원을 할 경우 출원보상을 한다. 보상금은 그 중요도에 따라 건당 10만 원에서 30만 원을 지급하며, 나머지는 회사 내에 설치된 직무발명심의위원회의 결정 심사 후에 지급한다.
(2)회사 명의로 등록된 특허권을 매도 또는 임대하였을 때 처분보상을 한다. 특허권을 타인에게 유상으로 임대한 경우에는 특허임대수익의 5～10%에 해당하는 금액을 발명자에게 보상금으로 지급한다.
(3)단, 특허임대수익의 산정은 수령하였거나 수령할 총 임대료에서 개발비용, 영업비용을 제외한다.

<사실관계>
X는 Y사에 직무발명에 대한 정당한 보상금의 지급을 요청하는 소송을 하기 위해 법무법인에 찾아갔다. X는 Y사에서 2007년부터 4년 4개월 동안 연구원으로 근무하였다. X는 Y사의 다른 연구원들과 함께 A사가 독점하고 있는 의약품과 동등하면서도 제조원가 대비 38%로 생산 가능한 방법을 48억 원의 비용을 들여 발명하였다. 해당 발명에 관여한 연구원들은 Y사에게 직무발명에 관한 특허 받을 권리를 이전하였고, Y사는 자사명의로 특허출원을 하였다. 이와 관련된 특허발명에서 X의 기여도는 1/3로 인정받았고 출원보상은 상여금 명목으로 5천만 원을 지급받았다.
한편 Y사는 2010년도에 A사와 특허권 임대계약을 체결하기 위해 42억 원의 비용을 소요하였다. 그 계약에 의해 A사로부터 초회 대금 45억 원, 중간 정산대금 23억 원을 지급받음과 동시에 40개월 간의 임대료로 요율 3～5%에 의해 산정된 금액 24억 원을 수령하였다. 계약기간인 2030년까지 추가적으로 수령할 Y사의 임대료는 약 28억 원으로 추정된다.

본 특허로 얻을 Y사의 임대료 수익은 임대료 명목으로 지급받은 52억 원입니다. 따라서 최소 2억 6천만 원의 보상금을 청구할 수 있을까요?''',
    '''<규정>
(1)CCTV란 일정한 공간에 지속적으로 설치되어 사람 또는 사물의 영상을 촬영하는 장치이다.
(2)누구든지 CCTV를 설치․운영할 수 있으나, 공개된 장소에서의 설치․운영은 범죄의 예방 및 수사를 위하여 필요한 경우에만 가능하다.
(3)CCTV를 설치․운영하는 자는 CCTV를 설치하여 운영하고 있다는 내용을 알리는 CCTV 설치․운영 안내판을 설치하여야 한다.

<견해>
갑:택시 안은 공개된 장소가 아니다.
을:일정한 공간에 지속적으로 설치되어 사람의 영상을 촬영하는 휴대전화 카메라는 CCTV에 해당한다.
병:비공개된 자동차 내부에 설치․운영되며, 외부를 촬영하고 있는 블랙박스도 CCTV에 해당한다.

갑에 따르면, 택시 안에서는 CCTV 설치․운영 안내판을 설치하기만 하면 언제든지 CCTV를 설치․운영할 수 있을까?
''',
    '''<제도>
온실가스 배출권 거래 제도는 정부가 온실가스 배출 총량을 정해 온실가스를 배출하는 사업장에 연 단위 배출권을 할당하고, 사업장이 할당 범위 안에서 온실가스를 배출하거나 과부족 분량을 다른 사업장과 거래할 수 있도록 한 제도이다. 총량 설정을 통해 온실가스 배출량을 줄이되 거래 제도를 이용하여 효율성을 극대화한 것이 이 제도의 특징이다.

<사례>
갑국에는 온실가스를 연간 5단위씩 배출해 오던 기업 A와 B가 있는데 정부가 연간 배출권을 각각 2단위씩 할당했다. 즉 A와 B가 할당된 배출권대로 온실가스를 감축하면 각각 3단위씩 감축해야 한다. A와 B는 온실가스 배출량을 감축하는 설비를 갖추고 있고, 온실가스 배출량 한 단위를 감축하는 비용은 감축량에 정비례한다. A의 경우 첫째 단위 감축 비용은 2가 들지만 둘째 단위 감축 비용은 4가 들어, 단위가 늘어날 때 단위당 감축 비용은 2씩 증가한다. B의 경우 첫째 단위 감축 비용은 4가 들지만 둘째 단위 감축 비용은 8이 들어 4씩 증가한다. A, B 모두 감축 비용이 소요됨에도 불구하고 조업 수준은 유지하고자 한다.
배출권 거래는 한 번에 한 단위씩 A, B 사이에서만 가능하다고 하자. 거래가 성립하려면 A와 B 모두에게 이득이 될 수준에서 가격이 형성되어야 한다. 예컨대, A는 배출권 한 단위의 거래 가격이 배출량을 한 단위 더 감축하는 비용보다 높으면 파는 것이 이득이 되고, B는 구입한 배출권 덕분에 감축하지 않아도 되는 한 단위의 감축 비용보다 거래 가격이 낮으면 사는 것이 이득이 된다.

배출권 거래 가격이 10이라면 1단위 거래가 성립할 수 있는가?''',
    '''수십 명의 직원이 근무하는 정보국에는 A, B, C 세 부서가 있고, 각 부서에 1명 이상이 소속되어 있다. 둘 이상의 부서에 소속된 직원은 없다. 이들 직원의 감시와 관련하여 세 가지 사실이 알려져 있다.
(1)A의 모든 직원은 B의 어떤 직원을 감시한다. 이는 A 부서에 속한 직원은 누구나 B 부서 소속의 직원을 1명 이상 감시하고 있음을 의미한다.
(2)B의 모든 직원이 감시하는 C의 직원이 있다. 이는 C 부서의 직원 가운데 적어도 한 사람은 B 부서 모든 직원의 감시 대상임을 의미한다.
(3)C의 어떤 직원은 A의 모든 직원을 감시한다. 이는 C 부서에 속한 직원 가운데 적어도 한 사람은 A 부서의 모든 직원을 감시 대상으로 삼고 있음을 의미한다.

<원리>
갑이 을을 감시하고 을이 병을 감시하면, 갑은 병을 감시하는 것이다.

B의 어떤 직원은 A의 모든 직원을 감시하고 있는가?''',
    '''8개의 축구팀 A, B, C, D, E, F, G, H가 다음 단계 1～3에 따라 경기하였다.
단계 1:8개의 팀을 두 팀씩 1, 2, 3, 4조로 나눈 후, 각 조마다 같은 조에 속한 두 팀이 경기를 하여 이긴 팀은 준결승전에 진출한다.
단계 2:1조와 2조에서 준결승전에 진출한 팀끼리 경기를 하여 이긴 팀이 결승전에 진출하고, 3조와 4조에서 준결승전에 진출한 팀끼리 경기를 하여 이긴 팀이 결승전에 진출한다.
단계 3:결승전에 진출한 두 팀이 경기를 하여 이긴 팀이 우승한다.
무승부 없이 단계 3까지 마친 경기 결과에 대하여 갑, 을, 병, 정이 아래와 같이 진술하였다.
갑:A는 2승 1패였다.
을:E는 1승 1패였다.
병:C는 준결승전에서 B에 패했다.
정:H가 우승하였다.
그런데 이 중에서 한 명만 거짓말을 한 것으로 밝혀졌다.

을의 진술은 참인가?
'''
]

test['answer'] = [
    '''3.5''',
    '''20''',
    '''40400000''',
    '''True''',
    '''True''',
    '''False''',
    '''True''',
    '''False''',
    '''False''',
    '''True''',
]


for i in tqdm(range(10)):
    cur_question = test['question'][i]
    cur_answer = test['answer'][i]
    cur_model_input = cot_prompt.format(question=cur_question)
    if VERBOSE:
        print(cur_model_input)
        print("="*100)
    response = chain.invoke({"text":cur_model_input})
    result = response.content
    if "Error:" in result:
        print(result)
        break

    cur_prediction = parse_model_responses(result)
    if VERBOSE:
        # print("Raw response:", result)
        # print(response.response_metadata)
        print("Predicted answer:",cur_prediction)
        print("Reference answer:",cur_answer)

    if not cur_prediction:
        cur_prediction = result.split("####")[-1]

    pass_collected.append(cur_prediction.strip().replace("$", "0") == cur_answer)
    results_collected.append({"question": cur_question, "answer": cur_answer, "prediction": cur_prediction, "raw_response": result})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")

    if VERBOSE and i == 10:
        break

    with open("llama-70b_3shot_cot.json", "w") as f:
        json.dump(results_collected, f, indent=4)

  0%|          | 0/10 [00:00<?, ?it/s]

instruction: 주어진 문제를 읽고 예시를 참고해 풀이과정과 함께 답안을 서술하시오. 답안은 #### 으로 풀이와 구분하시오.

[Example 1]
Question:
컴퓨터 사용자 갑, 을, 병, 정의 아이디와 패스워드를 다음 규칙으로 정하고자 한다.

◦아이디는 apple, banana, cherry, durian 중 하나이다.
◦패스워드는 apple, banana, cherry, durian 중 하나이다.
◦하나의 아이디를 두 명 이상이 같이 쓸 수 없다.
◦하나의 패스워드를 두 명 이상이 같이 쓸 수 없다.
◦사용자의 아이디와 패스워드는 같을 수 없다.
◦을의 아이디는 cherry이다.
◦정의 패스워드는 durian이다.
◦병의 아이디는 아이디가 banana인 사용자의 패스워드와 같다.

정의 아이디는 apple인가?

Output:정의 아이디가 'apple'이라고 가정하자. 그렇다면 병의 아이디는 'durian'이 된다.
다른 조건에 따라 갑의 아이디는 'banana'가 되며, 갑의 패스워드는 'durian'이 된다.
그러나 이때 갑과 병의 패스워드가 일치하는 모순이 발생한다.
#### False

[Example 2]
Question:
A반 4명, B반 3명, C반 3명, D반 2명으로 구성된 동아리를 세 개의 팀으로 나누는데, 다음 조건을 만족한다.

◦각 학생은 어느 한 팀에만 포함된다.
◦각 팀은 최소한 3개의 반의 학생을 포함한다.
◦특정 반의 학생 전체를 포함한 팀은 없다.

각 팀의 학생의 수가 모두 같을 수 있는가?

Output:각 팀에 (a, a, b, c), (a. b. c. d). (a, b, c, d) 와 같이 배정할 수 있다. 이경우 각 팀의 학생수가 같다.
#### True

[Example 3]
Question:
<규정>
제1조주차수요를 유발하는 건축물 등(‘시설물’)을 건축하거나 설치하려는 자는 <표>의 용도별 설치기준에 따라 부설주차장을 설치하여야 한다.
제2조①부설주차장에 설치된 기계식주차장치가 노후ㆍ고장 등의

 10%|█         | 1/10 [00:01<00:17,  1.99s/it]

Results found: ['1', '', '', '', '', '', '', '', '2', '4', '', '3', '', '3', '', '2', '', '', '', '', '', '', '', '', '', '', '', '', '', '3', '3000', '3000', '100', '20', '2', '30', '10', '20', '20', '4', '3', '3', '', '', '4', '', '7', '7', '', '', '6', '3', '', '', '3', '', '9', '', '9', '', '', '6', '3', '', '', '3', '', '3', '3', '', '3', '', '', '1', '5', '1', '5']
Predicted answer: 5
Reference answer: 3.5
Acc: 0.0
instruction: 주어진 문제를 읽고 예시를 참고해 풀이과정과 함께 답안을 서술하시오. 답안은 #### 으로 풀이와 구분하시오.

[Example 1]
Question:
컴퓨터 사용자 갑, 을, 병, 정의 아이디와 패스워드를 다음 규칙으로 정하고자 한다.

◦아이디는 apple, banana, cherry, durian 중 하나이다.
◦패스워드는 apple, banana, cherry, durian 중 하나이다.
◦하나의 아이디를 두 명 이상이 같이 쓸 수 없다.
◦하나의 패스워드를 두 명 이상이 같이 쓸 수 없다.
◦사용자의 아이디와 패스워드는 같을 수 없다.
◦을의 아이디는 cherry이다.
◦정의 패스워드는 durian이다.
◦병의 아이디는 아이디가 banana인 사용자의 패스워드와 같다.

정의 아이디는 apple인가?

Output:정의 아이디가 'apple'이라고 가정하자. 그렇다면 병의 아이디는 'durian'이 된다.
다른 조건에 따라 갑의 아이디는 'banana'가 되며, 갑의 패스워드는 'durian'이 된다.
그러나 이때 갑과 병의 패스워드가 일치하는 모순이 발생한다.
#### False


 20%|██        | 2/10 [00:04<00:19,  2.38s/it]

...

Results found: ['10', '', '1', '2', '2', '4', '1', '', '1', '', '1', '2', '10', '1', '10', '2', '', '1', '4', '2', '8', '1', '', '1', '', '1', '4', '10', '1', '10', '4', '']
Predicted answer: 
Reference answer: False
Acc: 0.125
instruction: 주어진 문제를 읽고 예시를 참고해 풀이과정과 함께 답안을 서술하시오. 답안은 #### 으로 풀이와 구분하시오.

[Example 1]
Question:
컴퓨터 사용자 갑, 을, 병, 정의 아이디와 패스워드를 다음 규칙으로 정하고자 한다.

◦아이디는 apple, banana, cherry, durian 중 하나이다.
◦패스워드는 apple, banana, cherry, durian 중 하나이다.
◦하나의 아이디를 두 명 이상이 같이 쓸 수 없다.
◦하나의 패스워드를 두 명 이상이 같이 쓸 수 없다.
◦사용자의 아이디와 패스워드는 같을 수 없다.
◦을의 아이디는 cherry이다.
◦정의 패스워드는 durian이다.
◦병의 아이디는 아이디가 banana인 사용자의 패스워드와 같다.

정의 아이디는 apple인가?

Output:정의 아이디가 'apple'이라고 가정하자. 그렇다면 병의 아이디는 'durian'이 된다.
다른 조건에 따라 갑의 아이디는 'banana'가 되며, 갑의 패스워드는 'durian'이 된다.
그러나 이때 갑과 병의 패스워드가 일치하는 모순이 발생한다.
#### False

[Example 2]
Question:
A반 4명, B반 3명, C반 3명, D반 2명으로 구성된 동아리를 세 개의 팀으로 나누는데, 다음 조건을 만족한다.

◦각 학생은 어느 한 팀에만 포함된다.
◦각 팀은 최소한 3개의 반의 학생을 포함한다.
◦특정 반의 학생 전체를 포함한 팀은 없다.

각 팀의 학생의 수가 모두 같을 수 있는가?

Output

 90%|█████████ | 9/10 [01:59<00:17, 17.02s/it]

Results found: ['1', '', '', '', '', '', '', '', '2', '4', '', '3', '', '3', '', '2', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '3', '3000', '3000', '100', '20', '2', '30', '10', '20', '20', '4', '', '', '', '1', '1', '', '2', '', '3', '', '']
Predicted answer: 
Reference answer: False
Acc: 0.1111111111111111
instruction: 주어진 문제를 읽고 예시를 참고해 풀이과정과 함께 답안을 서술하시오. 답안은 #### 으로 풀이와 구분하시오.

[Example 1]
Question:
컴퓨터 사용자 갑, 을, 병, 정의 아이디와 패스워드를 다음 규칙으로 정하고자 한다.

◦아이디는 apple, banana, cherry, durian 중 하나이다.
◦패스워드는 apple, banana, cherry, durian 중 하나이다.
◦하나의 아이디를 두 명 이상이 같이 쓸 수 없다.
◦하나의 패스워드를 두 명 이상이 같이 쓸 수 없다.
◦사용자의 아이디와 패스워드는 같을 수 없다.
◦을의 아이디는 cherry이다.
◦정의 패스워드는 durian이다.
◦병의 아이디는 아이디가 banana인 사용자의 패스워드와 같다.

정의 아이디는 apple인가?

Output:정의 아이디가 'apple'이라고 가정하자. 그렇다면 병의 아이디는 'durian'이 된다.
다른 조건에 따라 갑의 아이디는 'banana'가 되며, 갑의 패스워드는 'durian'이 된다.
그러나 이때 갑과 병의 패스워드가 일치하는 모순이 발생한다.
#### False

[Example 2]
Question:
A반 4명, B반 3명, C반 3명, D반 2명으로 구성된 동아리를 세 개의 팀으로 나누는데, 다음 조건을 만족한다.

◦각 학생은 어느 

100%|██████████| 10/10 [02:18<00:00, 13.88s/it]

Results found: ['1', '', '', '', '', '', '', '', '2', '4', '', '3', '', '3', '', '2', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '3', '6000', '', '3000', '', '', '3000', '100', '30', '20', '', '30', '20', '2', '20', '20', '4', '8', '', '', '', '', '', '', '', '1', '3', '3', '', '', '', '2', '1', '1', '1', '', '', '']
Predicted answer: 
Reference answer: True
Acc: 0.1


In [ ]:
response

AIMessage(content="Let's solve the problem.\n\nQuestion: Richard lives in an apartment building with 15 floors. Each floor contains 8 units, and 3/4 of the building is occupied. What's the total number of unoccupied units In the building?\n\nOutput: Since 3/4 of the building is occupied, that means 1/4 of the building is unoccupied. There are 15 floors, and each floor has 8 units. So, the total number of units is 15 * 8 = 120. 1/4 of 120 is 120/4 = 30. So, there are 30 unoccupied units in the building.\n\n#### 30", response_metadata={'token_usage': {'completion_time': 0.164, 'completion_tokens': 139, 'prompt_time': 0.168, 'prompt_tokens': 695, 'queue_time': None, 'total_time': 0.332, 'total_tokens': 834}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-000859e0-1b38-4ca7-b893-08708d901d96-0')

##Let's apply Program-of-Thought prompting!

### Write your PoT prompt

In [14]:
pot_prompt = """Example 4를 해결하는 solver() 함수를 python으로 작성하시오.
[Example 1]
Question: 다음에서 추론한 것으로 옳은가?

컴퓨터 사용자 갑, 을, 병, 정의 아이디와 패스워드를 다음 규칙으로 정하고자 한다.

◦아이디는 apple, banana, cherry, durian 중 하나이다.
◦패스워드는 apple, banana, cherry, durian 중 하나이다.
◦하나의 아이디를 두 명 이상이 같이 쓸 수 없다.
◦하나의 패스워드를 두 명 이상이 같이 쓸 수 없다.
◦사용자의 아이디와 패스워드는 같을 수 없다.
◦을의 아이디는 cherry이다.
◦정의 패스워드는 durian이다.
◦병의 아이디는 아이디가 banana인 사용자의 패스워드와 같다.

정의 아이디는 apple 이다?
Output:
```python
def solver():
  jung = dict()
  byung = dict()
  eul = dict()
  gab = dict()

  jung['id'] = 'apple'
  eul['id'] = 'cherry'
  jung['pw'] = 'durian'
  byung['id'] = 'durian'
  gab['id'] = 'banana'
  gab['pw'] = 'durian'

  if jung['pw'] == gab['pw']:
    return False
  else:
    return True
```
[Example 2]
Question: A반 4명, B반 3명, C반 3명, D반 2명으로 구성된 동아리를 세 개의 팀으로 나누는데, 다음 조건을 만족한다.

◦각 학생은 어느 한 팀에만 포함된다.
◦각 팀은 최소한 3개의 반의 학생을 포함한다.
◦특정 반의 학생 전체를 포함한 팀은 없다.

각 팀의 학생의 수가 모두 같을 수 있는가?

Output:
```python
def solver():
  one = ['a', 'b', 'c', 'd']
  two = ['a', 'b', 'c', 'd']
  three = ['a', 'a', 'b', 'c']
  if len(one) + len(two) + len(three) == 12:
    return True
  else:
    return False
```

[Example 3]
Question:<규정>
제1조주차수요를 유발하는 건축물 등(‘시설물’)을 건축하거나 설치하려는 자는 <표>의 용도별 설치기준에 따라 부설주차장을 설치하여야 한다.
제2조①부설주차장에 설치된 기계식주차장치가 노후ㆍ고장 등의 이유로 작동이 불가능하거나 안전상 철거가 불가피한 경우 이를 철거할 수 있다.
②시설물의 소유자는 제1항에 따라 기계식주차장치를 철거함으로써 제1조에 따른 부설주차장의 설치기준에 미달하게 되는 경우에는 부설주차장을 추가로 설치하여야 한다.
③구청장은 제1항에 따라 기계식주차장치를 철거하는 경우 <표>의 부설주차장 설치기준을 2분의 1로 완화하여야 한다.
④제3항에 의해 완화된 설치기준에 따라 부설주차장을 설치한 이후 해당 시설물이 증축되거나 부설주차장 설치기준이 강화되는 용도로 변경될 때에는 그 증축 또는 용도변경하는 부분에 대해서만 <표>의 부설주차장 설치기준을 적용한다.
제3조시설물의 용도를 변경하는 경우 용도변경 시점의 부설주차장 설치기준에 따라 변경 후 용도의 최소 주차대수를 갖추도록 부설주차장을 설치하여야 한다.
<표>

시설물의 용도 설치기준(최소 주차대수)
위락시설 시설면적 100㎡당 1대
판매시설 시설면적 150㎡당 1대

<사례>
갑은 판매시설로 사용되는 시설면적 6,000㎡의 시설물의 소유자이다. 40대를 수용하는 기존 기계식주차장치가 고장으로 작동이 불가능하자 갑은 이 기계식주차장치를 전부 철거하고, 구청장으로부터 부설주차장 주차기준을 2분의 1로 완화 적용받아 20대를 수용하는 부설주차장을 설치하였다. 갑은 이 시설물의 시설면적 중 3,000㎡를 위락시설로 용도변경하려 한다.

<규정>에 따라 <사례>의 갑이 추가로 갖추어야 할 최소 주차대수는?
Output:
```python
def solver():
    area = 3000
    car = area / 100 - 20 / 2

    return car
```

[Example 4]
Question: {question}
Output:
```python
"""


# pot_prompt = """Example 1를 해결하는 solver() 함수를 python으로 작성하시오.
# [Example 1]
# Question: {question}
# Output:
# # ```python
# """

## Implement the helper functions

In [16]:
!pip -q install func-timeout

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [17]:
import func_timeout


def parse_code_blocks(text):
    # Regular expression to find python code blocks enclosed within triple backticks
    regex = r"```python(.*?)```"
    matches = re.findall(regex, text, re.DOTALL)  # DOTALL to allow dot to match newlines
    return [match.strip() for match in matches]

def safe_execute(code_string: str, keys=None):
    exec(code_string)
    try:
        ans = eval("solver()")
        return ans
    except:
        return None


def synthesize_program(result: str, prefix: str) -> str:
    program = prefix
    for i, line in enumerate(result.split('\n')):
        if i == 0:
            program += line + '\n'
        else:
            if line.startswith('    '):
                program += line + '\n'
            else:
                break
    program += 'ans = solver()'
    return program

def floatify_ans(ans):
    ans = float(ans)
    return ans



In [18]:
from collections import Counter

VERBOSE=True
results_collected = []
pass_collected = []

test = {}
test['question'] = ['''
<규정>
(1)형벌 중 중형에는 다음 여섯 등급이 있다.

1등급 사형
2등급 노역 5년 후 3천 리 밖으로 유배
3등급 3천 리 밖으로 유배
4등급 2천 리 밖으로 유배
5등급 노역 3년 6개월
6등급 노역 3년

(2)사람을 때려 재물을 빼앗은 자는 3천 리 밖으로 유배한다.
(3)다른 사람의 범죄를 도운 자는 범죄를 저지른 자보다 한 등급을 감경하여 처벌한다.
(4)자신을 체포하려는 포졸을 때려 상해를 입힌 자의 형벌은 네 등급을 가중한다.
(5)탈옥한 자의 형벌은 세 등급을 가중한다.
(6)자수한 자의 형벌은 세 등급을 감경한다.
(7)1∼3등급에서 형을 감경하는 경우 3등급, 4등급은 하나의 등급으로 취급한다. 가령 2등급에서 두 등급을 감경하면 5등급이다.
(8)3∼6등급에서 형을 가중하는 경우 2등급이 상한이다.
(9)(3)∼(6)의 형벌 가중ㆍ감경 사유 중 두 개 이상에 해당하면, 해당 사유 모두를 (3), (4), (5), (6)의 순서대로 적용한다.
<사례>
갑이 을을 때려 재물을 빼앗는 동안 병은 갑을 위하여 망을 보아주었다. 도망쳐 숨어 지내던 병은 포졸 정의 눈에 띄어 체포될 위기에 처하자 그를 때려 상해를 입히고 달아났다. 이후 병은 관아에 자수하고 갇혀 있던 중 탈옥하였다.
<규정>에 따라 <사례>의 병이 받을 형벌은? 만약 답이 1년 6개월이라면 1.5, 1년이라면 1과 같이 십진수로 표현하시오.''',
    '''키패드에 0 부터 9 까지 숫자가 적혀있다.
숫자가 적혀 있는 두 사각형이 한 변을 서로 공유할 때 두 숫자가 ‘인접’한다고 하자. 서로 다른 6개의 숫자를 한 번씩만 사용하여 만든 암호에 대하여 다음 정보가 알려져 있다.
◦4와 인접한 숫자 중 두 개가 사용되었다.
◦6이 사용되었다면 9도 사용되었다.
◦8과 인접한 숫자 중 한 개만 사용되었다.

5, 6, 7 중에 사용된 숫자는 한 개인가?''',
    '''<X법>
제1조(재해 등에 대한 보상) 국가의 업무 수행 중에 부상을 입거나 사망하면 재해 보상금을 지급하고, 치료로 인하여 생업에 종사하지 못하면 그 기간 동안 휴업 보상금을 지급한다. 다만, 다른 법령에 따라 국가의 부담으로 같은 종류의 보상금을 받은 자에게는 그 보상금에 상당하는 금액은 지급하지 아니한다.
제2조(재해 보상금의 지급) ① 제1조에 따른 재해 보상금은 사망 보상금과 장애 보상금으로 구분하며, 그 지급액은 다음과 같다.
1.사망 보상금은 고용노동부에서 공표하는 전체 산업체 월평균임금총액(사망한 해의 전년도를 기준으로 한다)의 36배에 상당하는 금액
2.장애 보상금은 장애등급에 따라 다음과 같이 정한다.
가～마. 장애등급 1급～5급: (생략)
바. 장애등급 6급: 사망 보상금의
제3조(휴업 보상금의 지급) 제1조에 따른 휴업 보상금은 통계청이 매년 공표하는 도시 및 농가가계비를 평균한 금액(전년도를 기준으로 한다)의 100분의 60에 해당하는 금액을 월 30일을 기준(31일이 말일인 경우에도 같다)으로 하여 1일 단위로 계산한 금액에 치료로 인하여 생업에 종사하지 못한 기간의 일수를 곱한 금액으로 한다.

<사례>
자영업자 갑은 2016년 8월 예비군 훈련 중 자신의 과실 없이 사고로 부상을 입어 60일간의 입원 치료로 생업에 종사하지 못하였고, 장애등급 6급 판정을 받았다. 갑의 월평균 수입은 360만 원이고, 고용노동부에서 공표하는 전체 산업체 월평균임금총액은 2015년 240만 원, 2016년 250만 원이다. 통계청이 공표하는 도시 및 농가가계비를 평균한 금액은 2015년 월 100만 원, 2016년 월 120만 원이다. 한편, 갑은 위 부상과 관련하여 X법이 아닌 다른 법령에 따라 국가로부터 재해 보상금으로 400만 원을 지급받았다.
<X법>을 <사례>에 적용할 때 갑이 지급받을 수 있는 보상금의 총합은?''',
    '''어떤 학과의 졸업 예정자 갑～무에 대해 다음이 알려졌다.

◦취업을 한 학생은 졸업평점이 3.5 이상이거나 외국어 인증시험에 합격했다.
◦인턴 경력이 있는 학생들 중 취업박람회에 참가하지 않은 학생은 아무도 없었다.
◦졸업평점이 3.5 이상이고 취업박람회에 참가한 학생은 모두 취업을 했다.
◦외국어 인증시험에 합격하고 인턴 경력이 있는 학생들은 모두 취업을 했다.

인턴 경력이 있고 졸업평점이 3.5 이상인 무는 취업을 했는가?''',
    '''여러 개의 프로그램이 동시에 실행되면서 같은 작업을 수행하는 병렬 프로그래밍에서, 각 프로그램이 사용하는 데이터는 일정한 메모리 영역에 저장되고 공유된다. 프로그램 P1∼P4와 이들이 사용하는 메모리 영역 M1∼M4에 대하여 다음이 성립한다.

◦P1∼P4만이 실행되고 각 프로그램은 M1∼M4를 사용한다. 각 프로그램은 적어도 1개 이상의 메모리 영역을 사용하고 어떤 프로그램에 의해서도 사용되지 않는 메모리 영역은 없다.
◦메모리 영역은 M1∼M4의 순서대로 일렬로 연결되어 있다.
◦전체 프로그램이 사용하는 메모리 영역의 개수의 합은 최대 6이다.
◦어떤 프로그램도 연속되는 2개의 메모리 영역을 사용할 수 없다.
◦P1은 2개의 메모리 영역을 사용한다.
◦P2는 M2를 사용한다.
◦P4는 P2가 사용하는 메모리 영역을 1개 이상 공유한다.

M2가 3개의 프로그램에 의해서 사용될 수도 있는가?''',
    '''<규정>
종업원은 직무와 관련하여 한 발명에 대하여 기여도에 따라 다음과 같이 보상을 받을 권리를 가진다.
(1)회사가 직무발명에 대한 특허출원을 할 경우 출원보상을 한다. 보상금은 그 중요도에 따라 건당 10만 원에서 30만 원을 지급하며, 나머지는 회사 내에 설치된 직무발명심의위원회의 결정 심사 후에 지급한다.
(2)회사 명의로 등록된 특허권을 매도 또는 임대하였을 때 처분보상을 한다. 특허권을 타인에게 유상으로 임대한 경우에는 특허임대수익의 5～10%에 해당하는 금액을 발명자에게 보상금으로 지급한다.
(3)단, 특허임대수익의 산정은 수령하였거나 수령할 총 임대료에서 개발비용, 영업비용을 제외한다.

<사실관계>
X는 Y사에 직무발명에 대한 정당한 보상금의 지급을 요청하는 소송을 하기 위해 법무법인에 찾아갔다. X는 Y사에서 2007년부터 4년 4개월 동안 연구원으로 근무하였다. X는 Y사의 다른 연구원들과 함께 A사가 독점하고 있는 의약품과 동등하면서도 제조원가 대비 38%로 생산 가능한 방법을 48억 원의 비용을 들여 발명하였다. 해당 발명에 관여한 연구원들은 Y사에게 직무발명에 관한 특허 받을 권리를 이전하였고, Y사는 자사명의로 특허출원을 하였다. 이와 관련된 특허발명에서 X의 기여도는 1/3로 인정받았고 출원보상은 상여금 명목으로 5천만 원을 지급받았다.
한편 Y사는 2010년도에 A사와 특허권 임대계약을 체결하기 위해 42억 원의 비용을 소요하였다. 그 계약에 의해 A사로부터 초회 대금 45억 원, 중간 정산대금 23억 원을 지급받음과 동시에 40개월 간의 임대료로 요율 3～5%에 의해 산정된 금액 24억 원을 수령하였다. 계약기간인 2030년까지 추가적으로 수령할 Y사의 임대료는 약 28억 원으로 추정된다.

본 특허로 얻을 Y사의 임대료 수익은 임대료 명목으로 지급받은 52억 원입니다. 따라서 최소 2억 6천만 원의 보상금을 청구할 수 있을까요?''',
    '''<규정>
(1)CCTV란 일정한 공간에 지속적으로 설치되어 사람 또는 사물의 영상을 촬영하는 장치이다.
(2)누구든지 CCTV를 설치․운영할 수 있으나, 공개된 장소에서의 설치․운영은 범죄의 예방 및 수사를 위하여 필요한 경우에만 가능하다.
(3)CCTV를 설치․운영하는 자는 CCTV를 설치하여 운영하고 있다는 내용을 알리는 CCTV 설치․운영 안내판을 설치하여야 한다.

<견해>
갑:택시 안은 공개된 장소가 아니다.
을:일정한 공간에 지속적으로 설치되어 사람의 영상을 촬영하는 휴대전화 카메라는 CCTV에 해당한다.
병:비공개된 자동차 내부에 설치․운영되며, 외부를 촬영하고 있는 블랙박스도 CCTV에 해당한다.

갑에 따르면, 택시 안에서는 CCTV 설치․운영 안내판을 설치하기만 하면 언제든지 CCTV를 설치․운영할 수 있을까?
''',
    '''<제도>
온실가스 배출권 거래 제도는 정부가 온실가스 배출 총량을 정해 온실가스를 배출하는 사업장에 연 단위 배출권을 할당하고, 사업장이 할당 범위 안에서 온실가스를 배출하거나 과부족 분량을 다른 사업장과 거래할 수 있도록 한 제도이다. 총량 설정을 통해 온실가스 배출량을 줄이되 거래 제도를 이용하여 효율성을 극대화한 것이 이 제도의 특징이다.

<사례>
갑국에는 온실가스를 연간 5단위씩 배출해 오던 기업 A와 B가 있는데 정부가 연간 배출권을 각각 2단위씩 할당했다. 즉 A와 B가 할당된 배출권대로 온실가스를 감축하면 각각 3단위씩 감축해야 한다. A와 B는 온실가스 배출량을 감축하는 설비를 갖추고 있고, 온실가스 배출량 한 단위를 감축하는 비용은 감축량에 정비례한다. A의 경우 첫째 단위 감축 비용은 2가 들지만 둘째 단위 감축 비용은 4가 들어, 단위가 늘어날 때 단위당 감축 비용은 2씩 증가한다. B의 경우 첫째 단위 감축 비용은 4가 들지만 둘째 단위 감축 비용은 8이 들어 4씩 증가한다. A, B 모두 감축 비용이 소요됨에도 불구하고 조업 수준은 유지하고자 한다.
배출권 거래는 한 번에 한 단위씩 A, B 사이에서만 가능하다고 하자. 거래가 성립하려면 A와 B 모두에게 이득이 될 수준에서 가격이 형성되어야 한다. 예컨대, A는 배출권 한 단위의 거래 가격이 배출량을 한 단위 더 감축하는 비용보다 높으면 파는 것이 이득이 되고, B는 구입한 배출권 덕분에 감축하지 않아도 되는 한 단위의 감축 비용보다 거래 가격이 낮으면 사는 것이 이득이 된다.

배출권 거래 가격이 10이라면 1단위 거래가 성립할 수 있는가?''',
    '''수십 명의 직원이 근무하는 정보국에는 A, B, C 세 부서가 있고, 각 부서에 1명 이상이 소속되어 있다. 둘 이상의 부서에 소속된 직원은 없다. 이들 직원의 감시와 관련하여 세 가지 사실이 알려져 있다.
(1)A의 모든 직원은 B의 어떤 직원을 감시한다. 이는 A 부서에 속한 직원은 누구나 B 부서 소속의 직원을 1명 이상 감시하고 있음을 의미한다.
(2)B의 모든 직원이 감시하는 C의 직원이 있다. 이는 C 부서의 직원 가운데 적어도 한 사람은 B 부서 모든 직원의 감시 대상임을 의미한다.
(3)C의 어떤 직원은 A의 모든 직원을 감시한다. 이는 C 부서에 속한 직원 가운데 적어도 한 사람은 A 부서의 모든 직원을 감시 대상으로 삼고 있음을 의미한다.

<원리>
갑이 을을 감시하고 을이 병을 감시하면, 갑은 병을 감시하는 것이다.

B의 어떤 직원은 A의 모든 직원을 감시하고 있는가?''',
    '''8개의 축구팀 A, B, C, D, E, F, G, H가 다음 단계 1～3에 따라 경기하였다.
단계 1:8개의 팀을 두 팀씩 1, 2, 3, 4조로 나눈 후, 각 조마다 같은 조에 속한 두 팀이 경기를 하여 이긴 팀은 준결승전에 진출한다.
단계 2:1조와 2조에서 준결승전에 진출한 팀끼리 경기를 하여 이긴 팀이 결승전에 진출하고, 3조와 4조에서 준결승전에 진출한 팀끼리 경기를 하여 이긴 팀이 결승전에 진출한다.
단계 3:결승전에 진출한 두 팀이 경기를 하여 이긴 팀이 우승한다.
무승부 없이 단계 3까지 마친 경기 결과에 대하여 갑, 을, 병, 정이 아래와 같이 진술하였다.
갑:A는 2승 1패였다.
을:E는 1승 1패였다.
병:C는 준결승전에서 B에 패했다.
정:H가 우승하였다.
그런데 이 중에서 한 명만 거짓말을 한 것으로 밝혀졌다.

을의 진술은 참인가?
'''
]

test['answer'] = [
    '''3.5''',
    '''20''',
    '''40400000''',
    '''True''',
    '''True''',
    '''False''',
    '''True''',
    '''False''',
    '''False''',
    '''True''',
]


for i in tqdm(range(10)):
    #cur_question = gsm8k['question'][i]
    cur_question = test['question'][i]
    #cur_answer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_answer = test['answer'][i]
    cur_model_input = pot_prompt.format(question=cur_question)
    if VERBOSE:
        print(cur_model_input)
        print("="*100)
    response = chain.invoke({"text":cur_model_input})
    result = response.content
    if "Error:" in result:
        print(result)
        break

    # Implement code for processing the generated code
    result_counter = Counter()
    codes = parse_code_blocks(result)
    for r in codes:
        ans = safe_execute(r)
        if ans is not None:
            result_counter.update([ans])

    if len(result_counter) > 0:
        prediction = result_counter.most_common(1)[0][0]
    else:
        prediction = None
    cur_prediction =  prediction
    if type(cur_prediction) == int:
        cur_prediction = float(cur_prediction)

    if VERBOSE:
        print("Raw response:", result)
        print(response.response_metadata)
        print("Predicted answer:",cur_prediction, type(cur_prediction))
        print("Reference answer:",cur_answer, type(cur_answer))

    # if not cur_prediction:
    #     cur_prediction = result.split("####")[-1]

    # print(cur_answer.isalpha())
    if cur_answer.isalpha():
        pass_collected.append(cur_prediction == (cur_answer == 'True'))
    else:
        pass_collected.append(cur_prediction == float(cur_answer))
    results_collected.append({"question": cur_question, "answer": cur_answer, "prediction": cur_prediction, "raw_response": result})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")

    if VERBOSE and i == 10:
        break

    with open("result_3shot_pot.json", "w") as f:
        json.dump(results_collected, f, indent=4)

  0%|          | 0/10 [00:00<?, ?it/s]

Example 1를 해결하는 solver() 함수를 python으로 작성하시오.
[Example 1]
Question: 
<규정>
(1)형벌 중 중형에는 다음 여섯 등급이 있다.

1등급 사형
2등급 노역 5년 후 3천 리 밖으로 유배
3등급 3천 리 밖으로 유배
4등급 2천 리 밖으로 유배
5등급 노역 3년 6개월
6등급 노역 3년

(2)사람을 때려 재물을 빼앗은 자는 3천 리 밖으로 유배한다.
(3)다른 사람의 범죄를 도운 자는 범죄를 저지른 자보다 한 등급을 감경하여 처벌한다.
(4)자신을 체포하려는 포졸을 때려 상해를 입힌 자의 형벌은 네 등급을 가중한다.
(5)탈옥한 자의 형벌은 세 등급을 가중한다.
(6)자수한 자의 형벌은 세 등급을 감경한다.
(7)1∼3등급에서 형을 감경하는 경우 3등급, 4등급은 하나의 등급으로 취급한다. 가령 2등급에서 두 등급을 감경하면 5등급이다.
(8)3∼6등급에서 형을 가중하는 경우 2등급이 상한이다.
(9)(3)∼(6)의 형벌 가중ㆍ감경 사유 중 두 개 이상에 해당하면, 해당 사유 모두를 (3), (4), (5), (6)의 순서대로 적용한다.
<사례>
갑이 을을 때려 재물을 빼앗는 동안 병은 갑을 위하여 망을 보아주었다. 도망쳐 숨어 지내던 병은 포졸 정의 눈에 띄어 체포될 위기에 처하자 그를 때려 상해를 입히고 달아났다. 이후 병은 관아에 자수하고 갇혀 있던 중 탈옥하였다.
<규정>에 따라 <사례>의 병이 받을 형벌은? 만약 답이 1년 6개월이라면 1.5, 1년이라면 1과 같이 십진수로 표현하시오.
Output:
# ```python



 10%|█         | 1/10 [00:01<00:17,  1.94s/it]

Raw response: Here is the Python function `solver()` that solves the problem:
```python
def solver():
    # Initialize the penalty level
    penalty_level = 3  # 3천 리 밖으로 유배 (according to rule (2))

    # Apply rule (3) - 도운 자는 범죄를 저지른 자보다 한 등급을 감경
    penalty_level -= 1

    # Apply rule (4) - 자체포하려는 포졸을 때려 상해를 입힌 자의 형벌은 네 등급을 가중
    penalty_level += 4

    # Apply rule (5) - 탈옥한 자의 형벌은 세 등급을 가중
    penalty_level += 3

    # Apply rule (6) - 자수한 자의 형벌은 세 등급을 감경
    penalty_level -= 3

    # Adjust the penalty level according to rule (7)
    if penalty_level <= 3:
        penalty_level = max(1, penalty_level)

    # Convert the penalty level to the corresponding sentence
    sentences = {
        1: "사형",
        2: "노역 5년 후 3천 리 밖으로 유배",
        3: "3천 리 밖으로 유배",
        4: "2천 리 밖으로 유배",
        5: "노역 3년 6개월",
        6: "노역 3년"
    }
    sentence = sentences.get(penalty_level, "Unknown")

    # Convert the sentence to a numerical value (if applicable)
    if sentence.startswith("노역

 20%|██        | 2/10 [00:03<00:15,  1.88s/it]

...

A can benefit from selling one unit of emission right
B can benefit from buying one unit of emission right
The trade can occur at a price of 10
A can benefit from selling one unit of emission right
B can benefit from buying one unit of emission right
The trade can occur at a price of 10
Raw response: Here is the Python code to solve the problem:
```python
def solver():
    # Define the cost functions for A and B
    def cost_A(x):
        return 2 * x

    def cost_B(x):
        return 4 * x

    # Define the initial allocation of emission rights
    allocation_A = 2
    allocation_B = 2

    # Define the total emission reduction required
    total_reduction = 3

    # Calculate the cost of reducing emissions for A and B
    cost_A_reduce = cost_A(total_reduction)
    cost_B_reduce = cost_B(total_reduction)

    # Calculate the cost of buying/selling one unit of emission right
    trade_price = 10

    # Check if the trade is beneficial for A
    if trade_price > cost_A_reduce:
       

 90%|█████████ | 9/10 [00:20<00:02,  2.96s/it]

False
Raw response: Here is the Python solver function for Example 1:
```python
def solver():
    # Let's represent the surveillance relationships between departments
    # using a dictionary where each key is a department and the value is a set of departments it surveils
    surveillance = {
        'A': {'B'},  # A surveils B
        'B': {'C'},  # B surveils C
        'C': {'A'}   # C surveils A
    }

    # We want to check if some B employee surveils all A employees
    # Let's see if there's a B employee who surveils someone in A
    for b_employee in surveillance['B']:
        if b_employee in surveillance['C'] and 'A' in surveillance[b_employee]:
            return True  # Found a B employee who surveils all A employees

    return False  # No B employee surveils all A employees

print(solver())  # Output: True
```
Here's the explanation:

We represent the surveillance relationships between departments using a dictionary `surveillance`. Each key is a department, and the value i

100%|██████████| 10/10 [00:22<00:00,  2.22s/it]

True
Raw response: Here is a Python solver function to determine whether 을's statement is true or not:
```python
def solver():
    # Define the teams and their results
    teams = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    results = {
        'A': (2, 1),  # 2 wins, 1 loss
        'E': (1, 1),  # 1 win, 1 loss
        'C': (0, 1),  # lost to B in semifinals
        'H': (2, 0)  # won the championship
    }

    # Define the possible outcomes for each team in each stage
    stage1_outcomes = {
        'A': ['win', 'win', 'win', 'win'],
        'B': ['win', 'win', 'win', 'win'],
        'C': ['win', 'win', 'win', 'loss'],
        'D': ['win', 'win', 'win', 'win'],
        'E': ['win', 'win', 'loss', 'loss'],
        'F': ['win', 'win', 'win', 'win'],
        'G': ['win', 'win', 'win', 'win'],
        'H': ['win', 'win', 'win', 'win']
    }

    stage2_outcomes = {
        'A': ['win', 'loss'],
        'B': ['win', 'win'],
        'C': ['loss', 'loss'],
        'E': ['loss', 'loss'],
  

In [ ]:
T